In [11]:
!pip install -q --upgrade pip
!pip install -q docling qdrant-client[fastembed] tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [fastembed]


In [12]:
from google.colab import drive
from pathlib import Path

drive.mount('/content/drive')

OUT_DIR = Path('/content/drive/Shareddrives/Colab/docling-json')
JSONS   = sorted(OUT_DIR.glob('*.json'))
print("Found", len(JSONS), "Docling JSON files")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 12 Docling JSON files


In [13]:
from google.colab import userdata

In [14]:
EMBED_MODEL_ID = "sentence-transformers/all-MiniLM-L6-v2"
SPARSE_MODEL_ID = "Qdrant/bm25"
COLLECTION    = "medicare_policy_docs"
# QDRANT_DB     = '/content/drive/Shareddrives/Colab/medicare_policy_db/'

In [15]:
from qdrant_client import QdrantClient, models

client = QdrantClient(
    url=userdata.get('QDRANT_URL'),
    api_key=userdata.get('QDRANT_API_KEY')
)

client.set_model(EMBED_MODEL_ID) # dense
client.set_sparse_model(SPARSE_MODEL_ID) # sparse  (RRF merge)

print(client.get_collections())

# client = QdrantClient(path=QDRANT_DB)
# client.set_model(EMBED_MODEL_ID) # dense
# client.set_sparse_model(SPARSE_MODEL_ID) # sparse  (RRF merge)

# if COLLECTION not in [c.name for c in client.get_collections().collections]:
#   client.create_collection(
#     collection_name=COLLECTION,
#     vectors_config={
#       EMBED_MODEL_ID: models.VectorParams(
#       size=client.get_embedding_size(EMBED_MODEL_ID),
#       distance=models.Distance.COSINE
#     )
#     },  # size and distance are model dependent
#     sparse_vectors_config={SPARSE_MODEL_ID: models.SparseVectorParams()},
# )

collections=[]


In [16]:
from docling.document_converter import DocumentConverter
from docling.chunking import HybridChunker
from docling.datamodel.document import DoclingDocument
from docling.datamodel.base_models import InputFormat

In [17]:

for json_file in JSONS:

  doc = DoclingDocument.load_from_json(json_file)

  chunks, metas = [], []

  for chunk in HybridChunker(tokenizer=EMBED_MODEL_ID).chunk(doc):
    chunks.append(chunk.text)
    metas.append(chunk.meta.export_json_dict())

  _ = client.add(
    collection_name=COLLECTION,
    documents=chunks,
    metadata=metas,
    batch_size=64,
  )



tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (864 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (598 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3708 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2051 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2929 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for 